<img src="dask_horizontal.svg" 
     width="30%" 
     align=right
     alt="Dask logo">

Custom Workflows
------------------

We submit tasks directly to the task scheduler.  This demonstrates the flexibility that can be achieved with the `submit` function and normal Python for loops.

Later on we map functions across Python queues to construct data processing pipelines.

In [ ]:
from dask.distributed import Executor, progress
e = Executor('localhost:8786')
e

In [ ]:
from time import sleep

def inc(x):
    from random import random
    sleep(random())
    return x + 1

def double(x):
    from random import random
    sleep(random())
    return 2 * x
    
def add(x, y):
    from random import random
    sleep(random())
    return x + y 

### Submit many tasks

We submit many tasks that depend on each other in a normal Python for loop

In [ ]:
%%time
zs = []
for i in range(16):
    x = e.submit(inc, i)
    y = e.submit(double, x)
    z = e.submit(add, x, y)
    zs.append(z)

In [ ]:
e.gather(zs)

### Custom computation: Tree summation

As an example of a non-trivial algorithm, consider the classic tree reduction.  We accomplish this with a nested for loop and a bit of normal Python logic.

```
finish           total             single output
    ^          /        \
    |        c1          c2        neighbors merge
    |       /  \        /  \
    |     b1    b2    b3    b4     neighbors merge
    ^    / \   / \   / \   / \
start   a1 a2 a3 a4 a5 a6 a7 a8    many inputs
```

In [ ]:
L = zs
while len(L) > 1:
    new_L = []
    for i in range(0, len(L), 2):
        future = e.submit(add, L[i], L[i + 1])  # add neighbors
        new_L.append(future)
    L = new_L                                   # swap old list for new
    
progress(L)

In [ ]:
L

In [ ]:
e.gather(L)

Example with data streams
----------------------------

The executor can map functions over lists or queues.  This is nothing more than calling `submit` many times.  We can chain maps on queues together to construct simple data processing pipelines.

All of this logic happens on the client-side.  None of this logic was hard-coded into the scheduler.  This simple streaming system is a good example of the kind of system that becomes easy for users to build when given access to custom task scheduling.

In [ ]:
from queue import Queue
from threading import Thread

def multiplex(n, q, **kwargs):
    """ Convert one queue into several equivalent Queues
    
    >>> q1, q2, q3 = multiplex(3, in_q)
    """
    out_queues = [Queue(**kwargs) for i in range(n)]
    def f():
        while True:
            x = q.get()
            for out_q in out_queues:
                out_q.put(x)
    t = Thread(target=f)
    t.daemon = True
    t.start()
    return out_queues        

```
           ----inc---->
          /            \ 
in_q --> q              \_add__ results
          \             / 
           ---double-->/
```

In [ ]:
in_q = Queue()
q = e.scatter(in_q)

In [ ]:
in_q.put(1)
q.get()

In [ ]:
q_1, q_2 = multiplex(2, q)

inc_q = e.map(inc, q_1)
double_q = e.map(double, q_2)

add_q = e.map(add, inc_q, double_q)

out_q = e.gather(add_q)

In [ ]:
in_q.put(10)
out_q.get()

In [ ]:
from random import random

def feed(q):
    for i in range(10000):
        sleep(random())
        q.put(i)
        
t = Thread(target=feed, args=(q,))
t.daemon = True
t.start()

In [ ]:
out_q.qsize()